这个代码用于计算文章所需要的表格，表格包含：
1.SNR的相对降低
2.探测率的降低
3.参数估计精度的降低

In [1]:
import pandas as pd
import numpy as np
import waveform as wf
import fishermatrix as fm
import dataprocess as dp
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
plt.rcParams['font.family'] = 'Times new roman'  # 设置字体为 Arial
#plt.rcParams['font.size'] = 15

pi = np.pi

In [2]:
#那么先来计算最简单的SNR降低
data = pd.read_fwf('population_data/Klein16_Q3delays.txt', header=None)
len_pop = len(data.iloc[:,1])-1     #这一行给出popIII模型的catalog长度

event_rate_p3 = 2500
Index_p3 = []
SNR_threshold = 8

In [20]:
mass_ratio0 = np.arange(0.1,10,0.01)
for nn in range(len(mass_ratio0)):
    mass_ratio = mass_ratio0[nn]
    eta = mass_ratio/(1+mass_ratio)**2
    j = eta*(2*np.sqrt(3)-3.5171*eta+2.5763*eta**2)
    print(j)

0.2637213776281587
0.28306944221878966
0.30145611466600153
0.31894057991312585
0.3355774790874028
0.3514173185727
0.366506837827269
0.3808893404805016
0.3946049927082821
0.40769109241906626
0.42018231237338494
0.43211092000049306
0.4435069763607748
0.45439851642562556
0.4648117126029645
0.4747710232220406
0.4842993275021251
0.49341804836283676
0.5021472642864753
0.5105058113124175
0.5185113761283039
0.5261805811205429
0.5335290621560557
0.5405715397867243
0.5473218844965463
0.5537931765479219
0.5599977609269168
0.5659472978369242
0.5716528091451842
0.57712472114648
0.5823729039724729
0.5874067079430718
0.5922349971275254
0.5968661803572327
0.601308239909194
0.6055687580583686
0.6096549416786035
0.6135736450551232
0.6173313910565342
0.6209343908007856
0.6243885619373334
0.6276995456567764
0.6308727225293013
0.633913227264327
0.6368259624756346
0.639615611528939
0.6422866505422318
0.6448433596032076
0.6472898332626389
0.6496299903576109
0.6518675832140489
0.6540062062738643
0.65604930418

In [42]:
np.random.seed(20020421)
freq = np.arange(1e-5,1,1e-5)
num_full = 0
num_win = 0
tot_snrfull = 0.0
tot_snrwin = 0.0
for nn in tqdm(range(event_rate_p3)):

    rand_int = np.random.randint(len_pop)       #这一行生成catalog长度的随机整数
    red1,m11,m12 = data.iloc[rand_int,0],data.iloc[rand_int,1],data.iloc[rand_int,2]        #赋值红移，质量1，质量2，两个主要参数

    Mass1 = (m11+m12)*0.94
    if 1e5 <Mass1<1e8*5:
        ratio1 = m11/m12
        rd_ang = np.random.uniform(0,pi,3)
        random = np.random.uniform(0,1)
        param1 = [Mass1,ratio1,red1]

        #signal = wf.gap_sfa(freq,param1,rd_ang,random,'tj')
        fi = wf.fin(Mass1,ratio1)
        fe = wf.fout(Mass1,ratio1)
        freq_rd = np.arange(fi,fe,1e-5)
        SIG0 = wf.sfa(freq_rd,param1,rd_ang,'tq')
        snr_full = dp.SNR(SIG0,SIG0,wf.PSD_tq(freq_rd),1e-5)

        if snr_full>SNR_threshold:
            num_full+=1

            SIG = wf.gap_sfa(freq,param1,rd_ang,random,'tq')
            freq_rd = SIG[0]
            sig_win = SIG[1]
            snr_win = dp.SNR(sig_win,sig_win,wf.PSD_tq(freq_rd),1e-5)
            tot_snrfull = tot_snrfull + snr_full
            tot_snrwin = tot_snrwin + snr_win
            if snr_win<SNR_threshold:
                num_win+=1





100%|██████████| 2500/2500 [14:19<00:00,  2.91it/s]


In [43]:
rela_ratio = num_win/num_full
rela_snr = np.abs(tot_snrfull-tot_snrwin)/tot_snrfull
print('Q3 rela_ratio:',rela_ratio)
print('Q3 rela_snr:',rela_snr)
print(num_full)

Q3 rela_ratio: 0.03897338403041825
Q3 rela_snr: 0.12660141092397084
2104


In [21]:
#那么先来计算最简单的SNR降低
data = pd.read_fwf('population_data/Klein16_PopIII.txt', header=None)
len_pop = len(data.iloc[:,1])-1     #这一行给出popIII模型的catalog长度

event_rate_p3 = 55000
Index_p3 = []
SNR_threshold = 8

In [45]:
np.random.seed(20020421)
freq = np.arange(1e-5,1,1e-5)
num_full2 = 0
num_win2 = 0
tot_snrfull2 = 0.0
tot_snrwin2 = 0.0
for nn in tqdm(range(event_rate_p3)):

    rand_int = np.random.randint(len_pop)       #这一行生成catalog长度的随机整数
    red1,m11,m12 = data.iloc[rand_int,0],data.iloc[rand_int,1],data.iloc[rand_int,2]        #赋值红移，质量1，质量2，两个主要参数

    Mass1 = (m11+m12)*0.94
    if 1e5 <Mass1<1e8*5:
        ratio1 = m11/m12
        rd_ang = np.random.uniform(0,pi,3)
        random = np.random.uniform(0,1)
        param1 = [Mass1,ratio1,red1]

        #signal = wf.gap_sfa(freq,param1,rd_ang,random,'tj')
        fi = wf.fin(Mass1,ratio1)
        fe = wf.fout(Mass1,ratio1)
        freq_rd = np.arange(fi,fe,1e-5)
        SIG0 = wf.sfa(freq_rd,param1,rd_ang,'tq')
        snr_full = dp.SNR(SIG0,SIG0,wf.PSD_tq(freq_rd),1e-5)

        if snr_full>SNR_threshold:
            num_full2+=1

            SIG = wf.gap_sfa(freq,param1,rd_ang,random,'tq')
            freq_rd = SIG[0]
            sig_win = SIG[1]
            snr_win = dp.SNR(sig_win,sig_win,wf.PSD_tq(freq_rd),1e-5)
            tot_snrfull2 = tot_snrfull2 + snr_full
            tot_snrwin2 = tot_snrwin2 + snr_win
            if snr_win<SNR_threshold:
                num_win2+=1


100%|██████████| 55000/55000 [18:11<00:00, 50.38it/s] 


In [46]:
rela_ratio2 = num_win2/num_full2
rela_snr2 = np.abs(tot_snrfull2-tot_snrwin2)/tot_snrfull2
print('PopIII rela_ratio:',rela_ratio2)
print('PopIII rela_snr:',rela_snr2)
print(num_full2)

PopIII rela_ratio: 0.06206252916472235
PopIII rela_snr: 0.0659030350875565
2143


In [47]:
#那么先来计算最简单的SNR降低
np.random.seed(1234)
data = pd.read_fwf('population_data/Klein16_Q3nodelays.txt', header=None)
len_pop = len(data.iloc[:,1])-1     #这一行给出popIII模型的catalog长度

event_rate_p3 = 5000
Index_p3 = []
SNR_threshold = 8

In [48]:
np.random.seed(20020421)
freq = np.arange(1e-5,1,1e-5)
num_full3 = 0
num_win3 = 0
tot_snrfull3 = 0.0
tot_snrwin3 = 0.0
for nn in tqdm(range(event_rate_p3)):

    rand_int = np.random.randint(len_pop)       #这一行生成catalog长度的随机整数
    red1,m11,m12 = data.iloc[rand_int,0],data.iloc[rand_int,1],data.iloc[rand_int,2]        #赋值红移，质量1，质量2，两个主要参数

    Mass1 = (m11+m12)*0.94
    if 1e5 <Mass1<1e8*5:
        ratio1 = m11/m12
        rd_ang = np.random.uniform(0,pi,3)
        random = np.random.uniform(0,1)
        param1 = [Mass1,ratio1,red1]

        #signal = wf.gap_sfa(freq,param1,rd_ang,random,'tj')
        fi = wf.fin(Mass1,ratio1)
        fe = wf.fout(Mass1,ratio1)
        freq_rd = np.arange(fi,fe,1e-5)
        SIG0 = wf.sfa(freq_rd,param1,rd_ang,'tq')
        snr_full = dp.SNR(SIG0,SIG0,wf.PSD_tq(freq_rd),1e-5)

        if snr_full>SNR_threshold:
            num_full3+=1

            SIG = wf.gap_sfa(freq,param1,rd_ang,random,'tq')
            freq_rd = SIG[0]
            sig_win = SIG[1]
            snr_win = dp.SNR(sig_win,sig_win,wf.PSD_tq(freq_rd),1e-5)
            tot_snrfull3 = tot_snrfull3 + snr_full
            tot_snrwin3 = tot_snrwin3 + snr_win
            if snr_win<SNR_threshold:
                num_win3+=1


100%|██████████| 5000/5000 [21:35<00:00,  3.86it/s]


In [49]:
rela_ratio3 = num_win3/num_full3
rela_snr3 = np.abs(tot_snrfull3-tot_snrwin3)/tot_snrfull3
num_full3 = num_full3
print('Q3_nod rela_ratio:',rela_ratio3)
print('Q3_nod rela_snr:',rela_snr3)
print(num_full3)

Q3_nod rela_ratio: 0.11145272867025365
Q3_nod rela_snr: 0.22800131806936003
2602
